# Obtaining robust cell population markers, and redefining/reassuring the biased cell populations

**TO RUN THIS NOTEBOOK YOU HAVE TO RUN 3H NOTEBOOK FULLY!!!**

## imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import scanpy as sc
import scanpy.external as sce
import pandas as pd
import numpy as np
import os
import triku as tk
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from tqdm.notebook import tqdm
import scipy.sparse as spr
import networkx as nx

In [ ]:
!pip install cellassign

In [ ]:
# local imports and imports from other notebooks
from cellassign import assign_cats
from fb_functions import make_gene_scoring_with_expr, plot_score_graph, plot_UMAPS_gene, plot_adata_cluster_properties, make_dicts_fraction_mean, plot_dotplot_gene
%store -r dict_colors_human
%store -r seed
%store -r magma
%store -r data_dir
%store -r dict_cats_fb_human

In [ ]:
mpl.rcParams['figure.dpi'] = 120
pd.options.display.float_format = "{:,.2f}".format

**IMPORTANT: I am running this analysis in a computer with ~500 GB of RAM. I will load many datasets at once, which might be too much for some computers. I took this decision conciously, to have as much info available at any time as possible. If you cannot run all the analysis at once, you can run it by parts.**

## Anndata loading

In [ ]:
ahlers_2022_dir = data_dir + '/ahlers_2022'
ahlers_2022_young_human_fb = sc.read(ahlers_2022_dir + '/ahlers_2022_young_human_fb_robust.h5')
ahlers_2022_young_human = sc.read(ahlers_2022_dir + '/ahlers_2022_young_human_processed.h5')

In [ ]:
boothby_2021_dir = data_dir + '/boothby_2021'
boothby_2021_ctrl_human_fb = sc.read(boothby_2021_dir + '/boothby_2021_ctrl_human_fb_robust.h5')
boothby_2021_ctrl_human = sc.read(boothby_2021_dir + '/boothby_2021_ctrl_human_processed.h5')

In [ ]:
deng_2021_dir = data_dir + '/deng_2021'
deng_2021_scar_fb = sc.read(deng_2021_dir + '/deng_2021_scar_fb_robust.h5')
deng_2021_scar = sc.read(deng_2021_dir + '/deng_2021_scar_processed.h5')

In [ ]:
gao_2021_dir = data_dir + '/gao_2021'
gao_2021_ctrl_human_fb = sc.read(gao_2021_dir + '/gao_2021_ctrl_human_fb_robust.h5')
gao_2021_ctrl_human = sc.read(gao_2021_dir + '/gao_2021_ctrl_human_processed.h5')

In [ ]:
gaydosik_2020_dir = data_dir + '/gaydosik_2020'
gaydosik_2020_ctrl_human_fb = sc.read(gaydosik_2020_dir + '/gaydosik_2020_ctrl_human_fb_robust.h5')
gaydosik_2020_ctrl_human = sc.read(gaydosik_2020_dir + '/gaydosik_2020_ctrl_human_processed.h5')

In [ ]:
gur_2022_dir = data_dir + '/gur_2022'
gur_2022_ctrl_human_fb = sc.read(gur_2022_dir + '/gur_2022_ctrl_human_fb_robust.h5')
gur_2022_ctrl_human = sc.read(gur_2022_dir + '/gur_2022_ctrl_human_processed.h5')

In [ ]:
he_2020_dir = data_dir + '/He_2020'
he_2020_ctrl_human_fb = sc.read(he_2020_dir + '/he_2020_ctrl_human_fb_robust.h5')
he_2020_ctrl_human = sc.read(he_2020_dir + '/he_2020_ctrl_human_processed.h5')

In [ ]:
hughes_2020_dir = data_dir + '/hughes_2020'
hughes_2020_ctrl_human_fb = sc.read(hughes_2020_dir + '/hughes_2020_ctrl_human_fb_robust.h5')
hughes_2020_ctrl_human = sc.read(hughes_2020_dir + '/hughes_2020_ctrl_human_processed.h5')

In [ ]:
kim_2020_dir = data_dir + '/Kim_2020'
kim_2020_ctrl_human_fb = sc.read(kim_2020_dir + '/kim_2020_ctrl_human_fb_robust.h5')
kim_2020_ctrl_human = sc.read(kim_2020_dir + '/kim_2020_ctrl_human_processed.h5')

In [ ]:
liu_2021_dir = data_dir + '/liu_2021'
liu_2021_ctrl_human_fb = sc.read(liu_2021_dir + '/liu_2021_ctrl_human_fb_robust.h5')
liu_2021_ctrl_human = sc.read(liu_2021_dir + '/liu_2021_ctrl_human_processed.h5')

In [ ]:
mariottoni_2021_dir = data_dir + '/mariottoni_2021'
mariottoni_2021_ctrl_human_fb = sc.read(mariottoni_2021_dir + '/mariottoni_2021_ctrl_human_fb_robust.h5')
mariottoni_2021_ctrl_human = sc.read(mariottoni_2021_dir + '/mariottoni_2021_ctrl_human_processed.h5')

In [ ]:
mirizio_2020_dir = data_dir + '/mirizio_2020'
mirizio_2020_scl_human_fb = sc.read(mirizio_2020_dir + '/mirizio_2020_scl_human_fb_robust.h5')
mirizio_2020_scl_human = sc.read(mirizio_2020_dir + '/mirizio_2020_scl_human_processed.h5')

In [ ]:
reynolds_2021_dir = data_dir + '/reynolds_2021'
reynolds_2021_ctrl_human_fb = sc.read(reynolds_2021_dir + '/reynolds_2021_ctrl_human_fb_robust.h5')
# Dataset with the rest of cells is not available. It can be used but since the dataset doesn't have good quality, is not fully relevant.

In [ ]:
rindler_2021_dir = data_dir + '/rindler_2021'
rindler_2021_ctrl_human_fb = sc.read(rindler_2021_dir + '/rindler_2021_ctrl_human_fb_robust.h5')
rindler_2021_ctrl_human = sc.read(rindler_2021_dir + '/rindler_2021_ctrl_human_processed.h5')

In [ ]:
sole_2020_dir = data_dir + '/Sole-Boldo_2020'
sole_2020_young_human_fb = sc.read(sole_2020_dir + '/sole_2020_young_human_fb_robust.h5')
sole_2020_young_human = sc.read(sole_2020_dir + '/sole_2020_young_human_processed.h5')

In [ ]:
tabib_2018_dir = data_dir + '/Tabib_2018'
tabib_2018_ctrl_human_fb = sc.read(tabib_2018_dir + '/tabib_2018_ctrl_human_fb_robust.h5')
tabib_2018_ctrl_human = sc.read(tabib_2018_dir + '/tabib_2018_ctrl_human_processed.h5')

In [ ]:
tabib_2021_dir = data_dir + '/Tabib_2021'
tabib_2021_ctrl_human_fb = sc.read(tabib_2021_dir + '/tabib_2021_ctrl_human_fb_robust.h5')
tabib_2021_ctrl_human = sc.read(tabib_2021_dir + '/tabib_2021_ctrl_human_processed.h5')

In [ ]:
theo_2020_dir = data_dir + '/Theocharidis_2020/'
theo_2020_ctrl_human_dm_fb = sc.read(theo_2020_dir + '/theo_2020_ctrl_human_dm_fb_robust.h5')
theo_2020_ctrl_human_dm = sc.read(theo_2020_dir + '/theo_2020_ctrl_human_dm_processed.h5')

In [ ]:
theo_2021_dir = data_dir + '/Theocharidis_2021/'
theo_2021_ctrl_human_fb = sc.read(theo_2021_dir + '/theo_2021_ctrl_human_fb_robust.h5')
theo_2021_ctrl_human = sc.read(theo_2021_dir + '/theo_2021_ctrl_human_processed.h5')

In [ ]:
vors_2020_dir = data_dir + '/Vorstandlechner_2020'
vors_2020_ctrl_human_fb = sc.read(vors_2020_dir + '/vors_2020_ctrl_human_fb_robust.h5')
vors_2020_ctrl_human = sc.read(vors_2020_dir + '/vors_2020_ctrl_human_processed.h5')

In [ ]:
vors_2021_dir = data_dir + '/Vorstandlechner_2021'
vors_2021_ctrl_human_fb = sc.read(vors_2021_dir + '/vors_2021_ctrl_human_fb_robust.h5')
vors_2021_ctrl_human = sc.read(vors_2021_dir + '/vors_2021_ctrl_human_processed.h5')

In [ ]:
xu_2021_dir = data_dir + '/xu_2021'
xu_2021_ctrl_human_fb = sc.read(xu_2021_dir + '/xu_2021_ctrl_human_fb_robust.h5')
xu_2021_ctrl_human = sc.read(xu_2021_dir + '/xu_2021_ctrl_human_processed.h5')

# Geting the definitive list of genes
This is not *really* definitive. We use the produced markers to later on, in the analysis table (not in notebooks) do a selection based on gene function, or pattern of expression. The *final* list is in the variable **genes** below.

In [ ]:
%store -r list_all_datasets_human
%store -r list_datasets_human

%store -r list_accepted_clusters_human
%store -r list_accepted_axis_human
%store -r list_names_human

In [ ]:
list_names_human

In [ ]:
dict_make_gene_scoring_cluster_robust_human = make_gene_scoring_with_expr(list_datasets=list_datasets_human, calculate_DEGs = True, group_name = 'cluster_robust', 
                                                     value_ref = 'scores', select_method = 'pval', 
                                                     list_clusters = list_accepted_clusters_human,
                                                     select_thres = 0.05)

In [ ]:
dict_make_gene_scoring_axis_robust_human = make_gene_scoring_with_expr(list_datasets=list_datasets_human, calculate_DEGs = True, group_name = 'axis_robust',
                                                     value_ref = 'scores', select_method = 'pval', select_thres = 0.05, 
                                                         list_clusters =list_accepted_axis_human,)

In [ ]:
%store dict_make_gene_scoring_cluster_robust_human
%store dict_make_gene_scoring_axis_robust_human

In [ ]:
dict_make_gene_scoring_cluster_robust_human['A1'].iloc[:40].sort_index()

In [ ]:
dict_make_gene_scoring_cluster_robust_human['A2'].iloc[:40].sort_index()

In [ ]:
dict_make_gene_scoring_cluster_robust_human['A3'].iloc[:40].sort_index()

In [ ]:
dict_make_gene_scoring_cluster_robust_human['A4'].iloc[:40].sort_index()

In [ ]:
dict_make_gene_scoring_cluster_robust_human['B1'].iloc[:40].sort_index()

In [ ]:
dict_make_gene_scoring_cluster_robust_human['B2'].iloc[:40].sort_index()

In [ ]:
dict_make_gene_scoring_cluster_robust_human['B3'].iloc[:40].sort_index()

In [ ]:
dict_make_gene_scoring_cluster_robust_human['B4'].iloc[:40].sort_index()

In [ ]:
dict_make_gene_scoring_cluster_robust_human['C1'].iloc[:40].sort_index()

In [ ]:
dict_make_gene_scoring_cluster_robust_human['C2'].iloc[:40].sort_index()

In [ ]:
dict_make_gene_scoring_cluster_robust_human['C3'].iloc[:40].sort_index()

In [ ]:
dict_make_gene_scoring_cluster_robust_human['C5'].iloc[:40].sort_index()

In [ ]:
dict_make_gene_scoring_cluster_robust_human['D1'].iloc[:40].sort_index()

In [ ]:
dict_make_gene_scoring_cluster_robust_human['D2'].iloc[:40].sort_index()

In [ ]:
dict_make_gene_scoring_cluster_robust_human['E1'].iloc[:40].sort_index()

In [ ]:
dict_make_gene_scoring_cluster_robust_human['T1'].iloc[:40].sort_index()

## Analyse the presence and expression pattern of marker genes (within fibroblast populations)

In this section we will plot UMAPs and dot plots of expression patterns of gene markers. This is useful because many gene markers tend to be markers of few clusters based on DE analysis but, in reality, they are also expressed in other clusters which might need to be considered.

In [ ]:
genes = sorted(list(set(['A2M', 'AADAC', 'ABCA10', 'ABCA8', 'ABCA9', 'ABCC9', 'ACAN', 'ACE', 'ACHE', 'ACKR3', 'ACKR4', 'ADAM12', 'ADAMTS18', 'ADAMTS4', 
                         'ADAMTSL5', 'ADGRE2', 'ADRA2A', 'AEBP1', 'AHRR', 'AKAP6', 'ALDH1A3', 'ALX4', 'ANGPTL5', 'ANGPTL7', 'ANKRD29', 'ANOS1', 'ANTXR2', 
                         'AOPEP', 'APCDD1', 'APOC1', 'APOD', 'APOE', 'AQP1', 'AQP3', 'ARHGAP15', 'ASPN', 'ATP1A2', 'AXIN2', 'BAMBI', 'BGN', 'BIRC3', 'BMP7', 
                         'BNC2', 'BTBD11', 'C11orf96', 'C19orf33', 'C1orf198', 'C1QTNF3', 'C2orf40/ECRG4', 'C3', 'C6', 'C7', 'C9orf3', 'CA12', 'CADM2', 'CAV2', 
                         'CCDC146', 'CCK', 'CCL19', 'CCL2', 'CD200R1', 'CD248', 'CD34', 'CD55', 'CD70', 'CD74', 'CD9', 'CDA', 'CDH11', 'CDKN1A', 'CENPW', 
                         'CERCAM', 'CES1', 'CFD', 'CFH', 'CFHR1', 'CH25H', 'CHADL', 'CHD1', 'CHN1', 'CHRDL1', 'CLDN1', 'CLEC14A', 'CLEC2A', 'CLEC3B', 'CLSTN3', 
                         'CNTN4', 'COCH', 'COL11A1', 'COL13A1', 'COL14A1', 'COL15A1', 'COL18A1', 'COL1A2', 'COL21A1', 'COL23A1', 'COL24A1', 'COL28A1', 'COL3A1', 
                         'COL5A2', 'COL6A1', 'COL6A2', 'COL6A3', 'COL6A5', 'COL8A1', 'COL9A3', 'COL9A3', 'COMP', 'CORIN', 'COX4I2', 'CPE', 'CPNE5', 'CPVL', 
                         'CRABP1', 'CRIP1', 'CRISPLD1', 'CSRP1', 'CTHRC1', 'CTSH', 'CTSK', 'CX3CL1', 'CXCL1', 'CXCL12', 'CXCL2', 'CXCL3', 'CYBRD1', 'CYGB', 
                         'CYP1B1', 'CYP26B1', 'CYP7B1', 'CYYR1', 'DBN1', 'DCN', 'DCXR', 'DIO2', 'DNAJA1', 'DOK6', 'DPEP1', 'DPP4', 'DPT', 'DUSP5', 'EBF2', 
                         'ECM1', 'EDIL3', 'EDNRA', 'EFEMP1', 'EGFLAM', 'EGR1', 'EGR2', 'ELL2', 'ELN', 'EMB', 'EMID1', 'EMP3', 'ENTPD2', 'ERRFI1', 'ETV1', 
                         'ETV4', 'EVA1A', 'F13A1', 'F2R', 'F2RL2', 'FABP3', 'FAM180B', 'FBLN1', 'FBLN2', 'FBN1', 'FBXO32', 'FGF7', 'FGFBP2', 'FIBIN', 'FMO1', 
                         'FMO2', 'FMO3', 'FOS', 'FOSL1', 'FOXC2', 'FOXS1', 'FXYD6', 'FZD1', 'GALNT15', 'GCH1', 'GDF10', 'GEM', 'GFRA2', 'GGT5', 'GLRB', 'GNAO1', 
                         'GPC3', 'GPM6B', 'GPNMB', 'GSN', 'HAPLN1', 'HAS2', 'HHIP', 'HLA-B', 'HLA-F', 'HMGA1', 'HMGCLL1', 'HOPX', 'HPSE2', 'HRH1', 'HSD3B7', 
                         'HSPB3', 'HTRA1', 'ICAM1', 'ICAM2', 'IER3', 'IGF1', 'IGFBP2', 'IGFBP3', 'IGFBP5', 'IGFBP6', 'IGFBP7', 'IL32', 'IL33', 'IL34', 'IL6', 
                         'INHBA', 'INMT', 'IRF1', 'IRF8', 'ISYNA1', 'ITGA6', 'ITGB4', 'ITIH5', 'ITM2A', 'JAK3', 'JUNB', 'KCNQ3', 'KDM6B', 'KIAA1217', 'KIF26B', 
                         'KLF5', 'KLK1', 'KLK4', 'KPNA2', 'KRT17', 'KRT19', 'LEF1', 'LEPR', 'LINC00327', 'LINC01133', 'LMO4', 'LMO7', 'LOX', 'LOXL2', 'LPAR4', 
                         'LRRC15', 'LRRC17', 'LRRN3', 'LSP1', 'LTBP2', 'MAB21L2', 'MAFF', 'MAGI1', 'MAP2', 'MAP7', 'MARCKSL1', 'MATN4', 'MCTP1', 'MDK', 'MEF2C', 
                         'MFAP5', 'MGP', 'MGST1', 'MKX', 'MME', 'MMP11', 'MMP16', 'MMP2', 'MRAS', 'MRPS6', 'MSC', 'MXRA5', 'MYO10', 'MYOC', 'NBL1', 'NDNF', 
                         'NECAB1', 'NFIB', 'NFIL3', 'NFKB1', 'NFKBIA', 'NGFR', 'NKD2', 'NLGN4X', 'NPTX2', 'NR2F2', 'NR4A3', 'NRP2', 'NTM', 'OGN', 'OLFML2A', 
                         'OSBP2', 'P2RY14', 'P3H2', 'P4HA3', 'PAFAH1B3', 'PALMD', 'PAMR1', 'PCDH19', 'PCOLCE2', 'PCSK1N', 'PCSK9', 'PDGFD', 'PDGFRL', 'PEAR1', 
                         'PGM2L1', 'PHACTR3', 'PI16', 'PIEZO2', 'PIM3', 'PKP4', 'PLA2G2A', 'PLA2G5', 'PLAUR', 'PLEKHA4', 'PLK2', 'PLPP5', 'PLXDC1', 'PMEPA1', 
                         'PNRC1', 'PODNL1', 'POSTN', 'PPARG', 'PPDPFL', 'PPIC', 'PPP1R14A', 'PPP1R15A', 'PPP1R15B', 'PRDM8', 'PRG4', 'PRKG2', 'PRSS23', 'PSAT1', 
                         'PTCH1', 'PTGDS', 'PTGIS', 'PTGS1/COX1', 'PTK7', 'PTN', 'PTPRD', 'PXDNL', 'QPCT', 'RAMP1', 'RANBP3L', 'RARRES2', 'RBFOX1', 'RBP4', 
                         'RBP5', 'REL', 'RGCC', 'RGS16', 'RHPN1', 'ROBO2', 'RSPO1', 'RSPO3', 'RSPO4', 'RUNX2', 'S100B', 'SBSPON', 'SCARA5', 'SCN4B', 'SCN7A', 
                         'SCRG1', 'SDC1', 'SDPR/CAVIN2', 'SEMA3B', 'SEMA3C', 'SEMA3E', 'SERPINA5', 'SERPINF1', 'SFRP1', 'SFRP2', 'SFRP4', 'SGCA', 'SGIP1', 
                         'SH3BGRL3', 'SHISA3', 'SHISAL1', 'SIX1', 'SLC22A16', 'SLC22A3', 'SLC26A7', 'SLC29A1', 'SLC2A1', 'SLC2A3', 'SLC5A3', 'SLC9A3R2', 
                         'SLCO2B1', 'SLIT2', 'SLITRK6', 'SLPI', 'SNAI2', 'SOCS3', 'SOD2', 'SOSTDC1', 'SOX8', 'SPARC', 'SPARCL1', 'SPON1', 'SPON2', 'SPRY1', 
                         'SPRY2', 'SPSB1', 'STC1', 'STC2', 'STMN1', 'STMN2', 'SULT1A1', 'SVEP1', 'TAC1', 'TAGLN', 'TBX3', 'TCEAL2', 'TCF7L2', 'TENM2', 'TENM3', 
                         'TFAP2A', 'TGFBI', 'THBS2', 'THBS4', 'THSD4', 'TIAM1', 'TIMP2', 'TIMP3', 'TM4SF1', 'TMEM150C', 'TMEM176A', 'TMEM52', 'TMTC2', 'TNC', 
                         'TNFAIP3', 'TNFAIP6', 'TNFRSF19', 'TNFSF13B', 'TNFSF14', 'TNMD', 'TNN', 'TNNC1', 'TNNT3', 'TNXB', 'TPD52', 'TPPP3', 'TRAC', 'TRIL', 
                         'TRPM3', 'TRPS1', 'TSPAN13', 'TSPAN8', 'TTR', 'TUBB4A', 'TWIST2', 'TXNIP', 'TYMP', 'UACA', 'UAP1', 'UBD', 'UGT3A2', 'VCAM1', 'VIPR2', 
                         'WFDC1', 'WIF1', 'WISP2 / CCN5', 'WNT10A', 'WNT2', 'WNT5A', 'WTAP', 'XG', 'YWHAH', 'ZC2HC1C', 'ZC3H12A', 'ZFP36'])))

genes = sorted(list(set(['ADAMTS9', 'ADGRD1', 'AIF1L', 'AKAP12', 'ANGPTL1', 'BCL11B', 'BHLHE40', 'C1S', 'CADM3', 'CAV1', 'CCDE80', 'CD151', 'CD200', 'CNN2', 'COL4A2',
                         'COL4A4', 'COL5A1', 'COL7A1', 'COL8A2', 'CPZ', 'DDIT4', 'DKK2', 'EDNRB', 'EFNB1', 'EMILIN2', 'FGL2', 'FMOD', 'FNDC1', 'GFPT2', 'GPX3', 'GREM2',
                         'HEG1', 'HMCN1', 'HPGD', 'ISLR', 'ISM1', 'ITGBL1', 'KCNK2', 'LSAMP', 'MAFB', 'MAMDC2', 'MATN2', 'MEOX2', 'METRNL', 'MFAP2', 'MFAP4', 'MMP27', 
                         'NCAM1', 'NDRG2', 'NRP1', 'OMD', 'PCOLCE', 'PHLDA1', 'PLTP', 'PODN', 'PREX1', 'PTGFR', 'PTMA', 'PTPRE', 'RAB32', 'SMOC2', 'SOX9', 'TBX15', 
                         'TCF4', 'THBS3', 'TMEM176B', 'TNFSF13B', 'UGDH', 'VASN'])))

genes = sorted(list(set(['COL8A1', 'ETV1', 'CAV2', 'ITGA6', 'ITGB4', 'KRT19', 'KLF5', 'NGFR', 'PEAR1', 'DUSP5', 'FOXS1'])))

To create the dot plot we will use two variables: the percentage of cells expressing the marker in each cluster, and the mean expression value of the expressing-cells in each cluster. To do that, we will create a df for each case and gene, compile the info for each adata and get the mean/std across the adatas. If one gene is not expressed in an adata, or the adata does not have a certain cluster, the value registered is a NaN, not zero. 

In [ ]:
plot_UMAPS_gene('cluster_robust', list_datasets=list_all_datasets_human, list_names=list_names_human)

In [ ]:
dict_fraction_cells, dict_mean_exp = make_dicts_fraction_mean(genes, list_all_datasets=list_all_datasets_human, list_accepted_clusters=list_accepted_clusters_human, 
                                                              list_names=list_names_human, clusterby='cluster_robust')

To do the dotplot we do some adjustments. We adjust the fraction of cells beign expressed to $x^{2/3}$ to make it slightly more visual. We also apply a alpha of 1 - std(fraction of cells) to account for markers that are expressed in disparily, so that they are visually not that relevant.


In [ ]:
for gene in genes:
    print(gene)
    plot_dotplot_gene(gene, dict_fraction_cells, dict_mean_exp)
    plot_UMAPS_gene(gene, list_datasets=list_all_datasets_human, list_names=list_names_human, n_cols=5)
    plt.show()

## Analyse the presence and expression pattern of marker genes (within all populations)

In this section we will plot UMAPs and dot plots of expression patterns of gene markers. This is useful because many gene markers tend to be markers of few clusters based on DE analysis but, in reality, they are also expressed in other clusters which might need to be considered.

In [ ]:
list_accepted_clusters_human_full = list(dict_cats_fb_human.keys())

In [ ]:
list_all_datasets_human_full = [ahlers_2022_young_human, boothby_2021_ctrl_human, 
                         deng_2021_scar, gao_2021_ctrl_human, gaydosik_2020_ctrl_human, gur_2022_ctrl_human, 
                         he_2020_ctrl_human, hughes_2020_ctrl_human, kim_2020_ctrl_human, 
                         liu_2021_ctrl_human, mariottoni_2021_ctrl_human,
                         mirizio_2020_scl_human, rindler_2021_ctrl_human,
                         sole_2020_young_human, tabib_2021_ctrl_human,
                         theo_2020_ctrl_human_dm, theo_2021_ctrl_human,
                         vors_2020_ctrl_human, vors_2021_ctrl_human, xu_2021_ctrl_human]

list_names_human_full = [adata.obs['Author'].values[0] + ' ' + str(int(adata.obs['Year'].values[0])) + ' human' for adata in list_all_datasets_human_full]

In [ ]:
plot_UMAPS_gene('assigned_cats', list_datasets=list_all_datasets_human_full, list_names=list_names_human_full)

In [ ]:
dict_fraction_cells_full, dict_mean_exp_full = make_dicts_fraction_mean(genes, list_all_datasets=list_all_datasets_human_full, list_names=list_names_human_full,
                                                                        list_accepted_clusters=list_accepted_clusters_human_full, clusterby='assigned_cats')

In [ ]:
for gene in genes:
    print(gene)
    plot_dotplot_gene(gene, dict_fraction_cells_full, dict_mean_exp_full, rotate=True)
    plot_UMAPS_gene(gene, list_datasets=list_all_datasets_human_full, list_names=list_names_human, n_cols=5)
    plt.show()